# ทดสอบการค้นคืนข่าวที่น่าชื่อถือบน anti fakenews ด้วย cosine similarity + TF-IDF (ใช้เฉพาะหัวข้อข่าว(header จาก anti_info.json) 10/12/2021)

In [1]:
import numpy as np
import nltk
import re
import pandas as pd 
import json
import os
import pickle

from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial

nltk.download('words')
th_stop = thai_stopwords()
en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

def split_word(text):
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if (not i in th_stop) & (not i in en_stop) & (not i in punctuation) & (not i in ["'",'"','“','”','‘','’','\n',"None", ' '])] 
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
def read_anti_refer():
    # Opening JSON file
    root_path = os.getcwd()
    path = os.path.join(root_path, '../Scrapy/Scrapy_project/spiders//fetch file/anti_info.json')
    f = open(path, encoding="utf8")
    
    # returns JSON object as
    # a dictionary
    anti_refer_text_data = json.load(f)

    # Closing file
    f.close()
    return anti_refer_text_data

In [3]:
anti_refer_text_data = read_anti_refer()

In [4]:
len(anti_refer_text_data)

2331

In [5]:
def get_anti_refer_info(anti_refer_text_data): #สำหรับดึงหัวข้อข่าวอ้างอิง (header) และลิงของข่าวอ้างอิง (link)
    refer_text_list = []

    for refer_text in range(len(anti_refer_text_data)):
        tmp = []
        tmp.append(anti_refer_text_data[refer_text]['header'])
        tmp.append(anti_refer_text_data[refer_text]['link'])
        refer_text_list.append(tmp)

    return refer_text_list

In [6]:
anti_refer_text_list = get_anti_refer_info(anti_refer_text_data)

In [7]:
anti_refer_text_list

[['ยกเลิกบัตรเอทีเอ็มหรือบัตรเดบิต ขอคืนค่าธรรมเนียมรายปีจากธนาคารได้ จริงหรือ?',
  'https://www.antifakenewscenter.com/%e0%b8%a2%e0%b8%81%e0%b9%80%e0%b8%a5%e0%b8%b4%e0%b8%81%e0%b8%9a%e0%b8%b1%e0%b8%95%e0%b8%a3%e0%b9%80%e0%b8%ad%e0%b8%97%e0%b8%b5%e0%b9%80%e0%b8%ad%e0%b9%87%e0%b8%a1%e0%b8%ab%e0%b8%a3%e0%b8%b7%e0%b8%ad%e0%b8%9a%e0%b8%b1%e0%b8%95%e0%b8%a3%e0%b9%80%e0%b8%94%e0%b8%9a%e0%b8%b4%e0%b8%95-%e0%b8%82%e0%b8%ad%e0%b8%84%e0%b8%b7%e0%b8%99%e0%b8%84%e0%b9%88%e0%b8%b2%e0%b8%98%e0%b8%a3%e0%b8%a3%e0%b8%a1%e0%b9%80%e0%b8%99%e0%b8%b5%e0%b8%a2%e0%b8%a1%e0%b8%a3%e0%b8%b2%e0%b8%a2%e0%b8%9b%e0%b8%b5%e0%b8%88%e0%b8%b2%e0%b8%81%e0%b8%98%e0%b8%99%e0%b8%b2%e0%b8%84%e0%b8%b2%e0%b8%a3%e0%b9%84%e0%b8%94%e0%b9%89-%e0%b8%88%e0%b8%a3%e0%b8%b4%e0%b8%87%e0%b8%ab%e0%b8%a3%e0%b8%b7%e0%b8%ad/'],
 ['ข่าวปลอม อย่าแชร์! เลขล็อค 2 ตัว และ 3 ตัว จากกองสลาก งวดวันที่ 16 พ.ย. 64',
  'https://www.antifakenewscenter.com/%e0%b8%82%e0%b9%88%e0%b8%b2%e0%b8%a7%e0%b8%9b%e0%b8%a5%e0%b8%ad%e0%b8%a1-%e0%b8%ad%e0%b8%a2%e0%b9%

In [8]:
len(anti_refer_text_list)

2331

## ------------------------------------------------------------------------------
## สร้าง dataframe ของ anti fake-news สำหรับอ้างถึงตอนแนบลิงค์ url แสดงผล

In [9]:
anti_refer_text = []
anti_refer_url = []
for i in range(len(anti_refer_text_list)):
    anti_refer_text.append(anti_refer_text_list[i][0]) #list ของส่วนเนื้อหาเท่านั้น
    anti_refer_url.append(anti_refer_text_list[i][1]) #list ของ url เท่านั้น


In [10]:
#ทำ list ให้เป็น dataframe
anti_original_text_and_headline_news_df = pd.DataFrame(list(zip(anti_refer_text, anti_refer_url)), columns=["หัวข้อข่าวอ้างอิงของ anti fake-news", "URL ของ anti fake-news"])

anti_original_text_and_headline_news_df

,หัวข้อข่าวอ้างอิงของ anti fake-news,URL ของ anti fake-news
0,ยกเลิกบัตรเอทีเอ็มหรือบัตรเดบิต ขอคืนค่าธรรมเน...,https://www.antifakenewscenter.com/%e0%b8%a2%e...
1,ข่าวปลอม อย่าแชร์! เลขล็อค 2 ตัว และ 3 ตัว จาก...,https://www.antifakenewscenter.com/%e0%b8%82%e...
2,ข่าวปลอม อย่าแชร์! พบเส้นทางพายุเคลื่อนตัวเข้า...,https://www.antifakenewscenter.com/%e0%b8%82%e...
3,ข่าวปลอม อย่าแชร์! พบธาตุเหล็กกับโครเมี่ยม ในว...,https://www.antifakenewscenter.com/%e0%b8%82%e...
4,ครม. อนุมัติการสร้างบ้านเช่า 1 แสนหลัง สำหรับผ...,https://www.antifakenewscenter.com/%e0%b8%84%e...
...,...,...
2326,“โรเบิร์ต สายควัน” วอนหยุดแชร์ข่าวปลอม ยันเลิก...,https://www.antifakenewscenter.com/%e0%b9%82%e...
2327,“ไอซ์ ปรีชญา” เดือด เว็บปลอมกุข่าว โดนรวบปาร์ต...,https://www.antifakenewscenter.com/%e0%b9%84%e...
2328,CPALL แจงเหตุข่าวนมเปรี้ยวยี่ห้อดังทำเด็กเสียช...,https://www.antifakenewscenter.com/cpall-%e0%b...
2329,แค่ข่าวลือ! วอนหยุดแชร์ภาพเก่าดินสไลด์ ปิดช่อง...,https://www.antifakenewscenter.com/%e0%b9%81%e...


In [11]:
anti_headline_tokens_list = [split_word(txt) for txt in anti_refer_text] #นำ list ของเนื้อหามาตัดคำ

In [12]:
anti_tokens_list_j = [','.join(tkn) for tkn in anti_headline_tokens_list]

----------------------------------------------------------------------------------
# สร้าง vocabulary set

In [13]:
print(anti_tokens_list_j[0].split(",")[0:2])


['ยกเลิก', 'บัตร']


In [14]:
## Create Vocabulary
tokens_list = []

for words in anti_tokens_list_j:
    # print(words)
    temp_list = words.split(",")
    # print(temp_list)
    for i in temp_list:
        tokens_list.append(i)

vocabulary = set(tokens_list)
vocabulary = list(vocabulary)

In [15]:
len(vocabulary)

4238

--------------------------------------------------------------------------------------

In [16]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
original_c_feat = tvec.fit_transform(anti_tokens_list_j)

### Save Trained TFID Model

In [17]:
### Save model
root_path = os.getcwd()
path = os.path.join(root_path, 'anti-tfid.pkl')
print(path)
with open(path,'wb') as handle:
    pickle.dump(original_c_feat, handle)

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\anti-tfid.pkl


In [18]:
### load model
root_path = os.getcwd()
path = os.path.join(root_path, 'anti-tfid.pkl')
print(path)
t = pickle.load(open(path,'rb'))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\anti-tfid.pkl


### Save Vocabulary

In [19]:
### Save Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_anti.txt')
print(path)
with open(path, "w", encoding="utf-8") as file:
    file.write(str(vocabulary))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_anti.txt


In [20]:
### load Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_anti.txt')
print(path)
with open(path, "r", encoding="utf-8") as file:
    data2 = eval(file.readline())

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_anti.txt


In [21]:
data2

['ทุกราย',
 '9',
 'Avigan',
 'ใส่',
 'แมส',
 'ดีขึ้น',
 'แฝงตัว',
 'เครื่องดื่ม',
 'จ่าย',
 'หัวใจ',
 'แห่งชาติ',
 'สีชมพู',
 'ตาฟาง',
 'แบบสอบถาม',
 'สับสน',
 'เบี้ย',
 'ทำความสะอาด',
 'รัฐประหาร',
 'FRESH',
 'ผู้เช่า',
 '0.5',
 'เทอม',
 'ตัวแทน',
 'กิโลกรัม',
 'ขยาย',
 'จิต',
 'มัย',
 '7.5',
 'เสียภาษี',
 'ช่าง',
 'กระทบ',
 'พังทลาย',
 'กาญจนบุรี',
 'ผิดคน',
 'การทำงาน',
 'Cash',
 'ซื่อ',
 'พี',
 'นันต๊ะ',
 'สแกน',
 'ตรุษจีน',
 'Life',
 'ตั้งครรภ์',
 'ลาย',
 'สอง',
 'ภาษาไทย',
 'Cancel',
 'Notebook',
 'ล่',
 'เผยแพร่',
 'ร่างทรง',
 'ประเมิน',
 'หอมแดง',
 'หมัก',
 'จำเป็นต้อง',
 'ต้อกระจก',
 'อยู่ร่วม',
 'หูหนวก',
 'โอสถ',
 'สำหรับ',
 'ประจำสัปดาห์',
 'เดบิต',
 'AFNC',
 'โห',
 'ฤดูกาล',
 'ข่าวด่วน',
 'Test',
 'lung',
 'เพจ',
 'ทดลอง',
 'อนุญาต',
 'น้ำร้อน',
 'สัตว์น้ำ',
 'ไก่',
 'ดอกบัว',
 'ตลาดเสรี',
 'ล้ม',
 'เดินทาง',
 'ช่อง',
 'มิจฉาชีพ',
 'ศาสนาอิสลาม',
 'ย้อน',
 'Flexadel',
 '23.00',
 'เซียลเซียส',
 'กัมพูชา',
 'อุบัติเหตุ',
 'โกน',
 'ห้างสรรพสินค้า',
 'เรียกค่าเสียหาย',
 'exe',

In [22]:
len(data2)

4238

## Create vector for Query/search keywords

In [23]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))
    
    x = tvec.transform(tokens)
    
    x = x.A[0]
    return x

### Calculate Cosine similarity of trained Tfidf to input query


In [28]:
query_vector = []
def cosine_similarity_T(k, query):
    tokens = split_word(str(query))
    q_df = pd.DataFrame(columns=['q_clean'])
    # print(tokens)
    q_df.loc[0,'q_clean'] =str(tokens)

    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    
    d_cosines = []
    
    query_vector = gen_vector_T(q_df['q_clean'])
    
    for d in original_c_feat.A:
        d_cosines.append(1 - spatial.distance.cosine(query_vector, d))
    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    d_cosines.sort()

    a = pd.DataFrame()
    for i in out:
        a.loc[i,'index'] = str(i)
        a.loc[i,'Subject'] = anti_original_text_and_headline_news_df['หัวข้อข่าวอ้างอิงของ anti fake-news'][i]

    d_cosines.sort(reverse=True)
    
    for j in range(k):
        a.loc[out[j],'Score'] = d_cosines[j]
        
    return a

In [38]:
%time anti_result = cosine_similarity_T(10,'มีกาแฟผสมยาอี  กำลังระบาดในกลุ่มวัยรุ่นมีกาแฟผสมยาอี กำลังระบาดในกลุ่มวัยรุ่น')

Wall time: 169 ms


In [39]:
anti_result

,index,Subject,Score
1723,1723,ข่าวปลอม อย่าแชร์! กาแฟผสมยาปลุกเซ็กซ์ เพิ่มสม...,0.326428
1767,1767,ข่าวปลอม อย่าแชร์! กาแฟผสมมะนาว น้ำมันมะพร้าว ...,0.282281
1398,1398,ข่าวปลอม อย่าแชร์! กาแฟผสมมะนาว น้ำมันมะพร้าว ...,0.255930
1480,1480,ข่าวปลอม อย่าแชร์! ดื่มกาแฟดำผสมมะนาว เเก้ปวดไ...,0.242430
1718,1718,ข่าวปลอม อย่าแชร์! กาแฟสารสกัดจากธรรมชาติ ต้าน...,0.202115
1414,1414,ข่าวบิดเบือน ไฮโดรเจนเปอร์ออกไซด์ผสมเบกกิ้งโซด...,0.188738
2114,2114,เตือน! โรคโมโนนิวคลิโอซิส ติดต่อจากการจูบ มีไข...,0.181153
255,255,“ผลิตภัณฑ์” ข่าวปลอมที่พุ่งสูงขึ้นในช่วงการระบ...,0.178954
1535,1535,ข่าวบิดเบือน ผงเบกกิ้งโซดาผสมน้ำ ใช้เป็นยาลดกร...,0.174342
2298,2298,‘ดีอี’ ยันไม่มีประกาศพื้นที่ควบคุมพิเศษ วอนตรว...,0.169305


In [40]:
anti_result_with_url = pd.DataFrame()
for i in range(len(anti_result)):
    anti_result_with_url.loc[i,'index'] = anti_result.iloc[i]["index"]
    anti_result_with_url.loc[i,'text'] = anti_result.iloc[i]["Subject"]
    anti_result_with_url.loc[i,'url'] = anti_original_text_and_headline_news_df["URL ของ anti fake-news"][int(anti_result.iloc[i]["index"])]

In [41]:
anti_result_with_url

,index,text,url
0,1723,ข่าวปลอม อย่าแชร์! กาแฟผสมยาปลุกเซ็กซ์ เพิ่มสม...,https://www.antifakenewscenter.com/%e0%b8%82%e...
1,1767,ข่าวปลอม อย่าแชร์! กาแฟผสมมะนาว น้ำมันมะพร้าว ...,https://www.antifakenewscenter.com/%e0%b8%82%e...
2,1398,ข่าวปลอม อย่าแชร์! กาแฟผสมมะนาว น้ำมันมะพร้าว ...,https://www.antifakenewscenter.com/%e0%b8%82%e...
3,1480,ข่าวปลอม อย่าแชร์! ดื่มกาแฟดำผสมมะนาว เเก้ปวดไ...,https://www.antifakenewscenter.com/%e0%b8%82%e...
4,1718,ข่าวปลอม อย่าแชร์! กาแฟสารสกัดจากธรรมชาติ ต้าน...,https://www.antifakenewscenter.com/%e0%b8%82%e...
5,1414,ข่าวบิดเบือน ไฮโดรเจนเปอร์ออกไซด์ผสมเบกกิ้งโซด...,https://www.antifakenewscenter.com/%e0%b8%82%e...
6,2114,เตือน! โรคโมโนนิวคลิโอซิส ติดต่อจากการจูบ มีไข...,https://www.antifakenewscenter.com/%e0%b9%80%e...
7,255,“ผลิตภัณฑ์” ข่าวปลอมที่พุ่งสูงขึ้นในช่วงการระบ...,https://www.antifakenewscenter.com/%e0%b8%9c%e...
8,1535,ข่าวบิดเบือน ผงเบกกิ้งโซดาผสมน้ำ ใช้เป็นยาลดกร...,https://www.antifakenewscenter.com/%e0%b8%82%e...
9,2298,‘ดีอี’ ยันไม่มีประกาศพื้นที่ควบคุมพิเศษ วอนตรว...,https://www.antifakenewscenter.com/%e0%b8%94%e...


---------------------------------------